In [2]:

!pip install scikit-learn-intelex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 KB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 48.0 MB/s eta 0:00:00


In [3]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [4]:
# --- Mounting drive --- 


from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# --- tqdm version 4.36.1 is required --- 


!pip install tqdm==4.36.1

In [ ]:
!pip install rogue

In [ ]:
# --- Import essential packages --- 


import numpy as np
import pandas as pd
import os
import pickle
import re
from sklearn.datasets import load_files
import glob
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from tqdm.auto import tqdm

tqdm.pandas()

In [5]:
# --- Implementing pretrained word embeddings --- 
from sklearnex import patch_sklearn
patch_sklearn()
from sklearnex import patch_sklearn
patch_sklearn()

from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec


glove_file = datapath("/content/drive/MyDrive/hackathon/test.txt")
tmp_file = get_tmpfile("/content/drive/MyDrive/hackathon/temp.txt")
converted_file = glove2word2vec(glove_file, tmp_file) 


# Source: https://radimrehurek.com/gensim/scripts/glove2word2vec.html

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [ ]:
# --- Loading the Glove embeddings in word2vec format ---

from sklearnex import patch_sklearn
patch_sklearn()


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [ ]:
glove_model["the"]  # Checking

In [ ]:
# --- Stop word removal function --- 
from sklearnex import patch_sklearn
patch_sklearn()

from spacy.lang.en.stop_words import STOP_WORDS

def stop_word_remove(sentence):
    temp = [token for token in sentence.split() if token not in STOP_WORDS]
    return ' '.join(word for word in temp)

In [ ]:
# --- Function to read the papers from their paths --- 

from sklearnex import patch_sklearn
patch_sklearn()
def read_paper(path):
  f = open(path, 'r', encoding="utf-8")
  text = str(f.read())
  f.close()
  return text

In [ ]:
# --- Function to preprocess the papers --- 

from sklearnex import patch_sklearn
patch_sklearn()
def process_paper(text):

  # Removes unwanted characters, accounting for unicode characters
  text = re.sub("@&#", " ", text)
  text = re.sub("\n", " ", text)
  text = (text.encode('ascii', 'ignore')).decode("utf-8")

  # Extracting the highlights, body from the paper
  highlights = re.findall(r'HIGHLIGHTS(.*?)KEYPHRASES', text,  flags = re.I)[0]
  body_main = re.findall(r'.*(?:abstract)(.*?)', text, flags=re.I)[0]

  # Making a copy of the body, lowercasing body text, removing punctuations & extra spaces
  dummy_body = body_main.lower()
  dummy_body = re.sub('[^\w\s\d\.]','',dummy_body)
  dummy_body = ' '.join(dummy_body.split())
  dummy_body = dummy_body.split(".")

  # Removing extra spaces from the body text, which will be preserved to produce summaries
  # And splitting into sentences
  body = ' '.join(body_main.split())
  body = body.split(".")

  # Removing sentences that are too short or too long, as they wouldn't make apt summary text
  for i,x in enumerate(dummy_body):
    if (len(x.split())) < 3 or (len(x.split())) > 15: 
        dummy_body.pop(i)
        body.pop(i)

  # Making a copy of the highlights, lowercasing body text, removing punctuations & extra spaces
  dummy_highlights = highlights.lower()
  dummy_highlights = re.sub('[^\w\s\d]','',dummy_highlights)
  dummy_highlights = ' '.join(dummy_highlights.split())

  # Removing stop words from body & highlights
  body_copy = []
  for x in dummy_body:
    body_copy.append(stop_word_remove(x))
  highlight_copy = []
  for x in dummy_highlights.split():
      highlight_copy.append(stop_word_remove(x))

  
  # Combing all of the highlights into one string    
  highlight_copy = " ".join(sentence for sentence in highlight_copy)
  highlight_copy = " ".join(highlight_copy.split())

  return body_main, body_copy, highlights, highlight_copy

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [ ]:
from scipy.special import expit


# Function to calculate sentence Score
def document_score(body_copy, highlight_copy):
  # Getting word vectors for the body
  body_vectors = []
  for sent in body_copy:
      sent_vec = []
      for word in sent.split():
          try:
              sent_vec.append(glove_model[word])
          # If the word vector isn't there in the model
          # then use the vector of the word "Visual"
          except:
              sent_vec.append(glove_model["visual"])
      body_vectors.append(sent_vec)

  # Getting word vectors for the highlights
  highlight_vectors = []
  for word in highlight_copy.split():
      try:
          highlight_vectors.append(glove_model[word])
      except:
          highlight_vectors.append(glove_model["visual"])

  # Finding the rouge score for each sentence by counting the # of common words
  # & dividing by length of sentence
  doc_score = []
  for sent in body_vectors:
      sent_score = 0
      for word in sent:
          for w in highlight_vectors:
              if (word == w).all():
                  sent_score+=1
      try: 
        doc_score.append(expit(sent_score/len(sent)))
      except:
        doc_score.append(0)
  return doc_score

In [ ]:
# Import libraries

from gensim.models import doc2vec
from collections import namedtuple


# Function to create document vectors
def create_document_vector(body_main, doc_score):
  # Load data
  doc1 = [body_main]

  # Transforming data
  docs = []
  analyzedDocument = namedtuple('AnalyzedDocument', 'words tags')
  for i, d in enumerate(doc1):
      words = d.lower().split()
      tags = [i]
      docs.append(analyzedDocument(words, tags))

  # Training model
  model = doc2vec.Doc2Vec(docs, size = 100, window = 300, min_count = 1, workers = 4)

  # Getting vectors
  doc_vec = model.docvecs[0]

  doc_vectors = []
  for i in range(len(doc_score)):
    doc_vectors.append(doc_vec)
  return doc_vectors

In [ ]:
# Function to create datasets
from sklearnex import patch_sklearn
patch_sklearn()
def create_data(path):
  text = read_paper(path)
  body_main, body_copy, highlights, highlight_copy = process_paper(text)
  doc_score = document_score(body_copy, highlight_copy)
  doc_vectors = create_document_vector(body_main, doc_score)
  sent_vectors = create_sentence_vectors(body_copy)
  x = np.concatenate([doc_vectors, sent_vectors], axis=1).tolist()
  x = pd.DataFrame(x)
  y = pd.DataFrame(doc_score)
  return x, y

In [ ]:
# --- Suppress all warnings --- 


import warnings
warnings.filterwarnings("ignore")



# --- Creating a list of all file paths & partially fitting the model --- 

paths = glob.glob("/content/drive/My Drive/NLP Project/Project Final/Parsed_Papers/*.txt")
for i,path in enumerate(tqdm(paths[0:20])):
  x, y = create_data(path)
  Model.partial_fit(x,y)

In [ ]:
f = open("/content/drive/MyDrive/hackathon/test.txt", 'r', encoding="utf-8")
text = str(f.read())
f.close()

In [6]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [ ]:
text = re.sub("@&#", " ", text)
text = re.sub("\n", " ", text)
text = (text.encode('ascii', 'ignore')).decode("utf-8")
text

In [ ]:
main_title = re.findall(r'MAIN-TITLE(.*?)HIGHLIGHTS', text, flags = re.I)
main_title

In [ ]:
highlights = re.findall(r'HIGHLIGHTS(.*?)KEYPHRASES', text,  flags = re.I)[0]
highlights

In [ ]:
keyphrases = re.findall(r'.*keyphrases(.*?)(?:introduction|abstract).*', text, flags = re.I)
keyphrases

In [ ]:
body_main = re.findall(r'.*(?:abstract)(.*?)references', text, flags=re.I)[0]
body_main

In [ ]:
dummy_body = body_main.lower()
dummy_body = re.sub('[^\w\s\d\.]','',dummy_body)
dummy_body = ' '.join(dummy_body.split())
dummy_body = dummy_body.split(".")
dummy_body[:5]

In [ ]:
body = ' '.join(body_main.split())
body = body.split(".")
body[:5]

# for x in body:
#     if x == " ":
#         body.remove(x)


In [ ]:
from sklearnex import patch_sklearn
patch_sklearn()
for i,x in enumerate(dummy_body):
  if (len(x.split())) < 3 or (len(x.split())) > 15: 
      dummy_body.pop(i)
      body.pop(i)
print(len(dummy_body),len(body))

In [ ]:
dummy_highlights = highlights.lower()
dummy_highlights = re.sub('[^\w\s\d]','',dummy_highlights)
dummy_highlights = ' '.join(dummy_highlights.split())
dummy_highlights

In [ ]:
body_copy = []

for x in dummy_body:
    body_copy.append(stop_word_remove(x))
        
body_copy[:5]

In [ ]:
highlight_copy = []

for x in dummy_highlights.split():
    highlight_copy.append(stop_word_remove(x))
    
highlight_copy = " ".join(sentence for sentence in highlight_copy)
highlight_copy = " ".join(highlight_copy.split())
highlight_copy

In [ ]:
body_vectors = []
from sklearnex import patch_sklearn
patch_sklearn()

for sent in body_copy:
    sent_vec = []
    for word in sent.split():
        try:
            sent_vec.append(glove_model[word])
        except:
            sent_vec.append(glove_model["visual"])
    body_vectors.append(sent_vec)
    
    
body_vectors[0]

In [ ]:
highlight_vectors = []


for word in highlight_copy.split():
    try:
        highlight_vectors.append(glove_model[word])
    except:
        highlight_vectors.append(glove_model["visual"])


highlight_vectors

In [ ]:
from scipy.special import expit
from sklearnex import patch_sklearn
patch_sklearn()

doc_score = []

for sent in body_vectors:

    sent_score = 0

    for word in sent:
        for w in highlight_vectors:
            if (word == w).all():
                sent_score+=1
    try: 
      doc_score.append(expit(sent_score/len(sent)))
    except:
      doc_score.append(0)

      
y = doc_score
y[:10]